# 處理房屋周遭環境資料(政府開放資料)

In [3]:
import pandas as pd
import geopandas as gpd
import csv
import re
from shapely.geometry import Point

import sys, os
sys.path.append("..")
import my_function.full_to_half as htf
import my_function.write_coordinate_to_csv as wctc
print('Pandas version: ', pd.__version__)
print('Geopandas version: ', gpd.__version__)

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "C:\Users\tazz4\anaconda3\envs\ox\lib\site-packages\fake_useragent\utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "C:\Users\tazz4\anaconda3\envs\ox\lib\site-packages\fake_useragent\utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


Pandas version:  1.4.3
Geopandas version:  0.11.1


In [ ]:
# 開放資料地址轉坐標
df = pd.read_csv(r'../rawdata/medical_facilities/pharmacy.csv', encoding = 'utf-8')
df.head()

In [ ]:
pattern = r'[0-9]*?[零一二三四五六七八九十]*?[0-9]*?[零一二三四五六七八九十]{1}[0-9]*?[號]'
df['地址'] = df['地址縣市別'] + df['地址鄉鎮市區'] + df['地址街道巷弄號']

In [ ]:
df.head()

In [ ]:
df['修正地址'] = df['地址'].str.contains(pattern,regex=True)
df.head()

In [ ]:
df['地址'] = htf.h2f(df['地址'])
df.to_csv('../rawdata/medical_facilities/pre_pharmacy.csv')

In [ ]:
wctc.write('../rawdata/medical_facilities/pre_pharmacy.csv', 'pharmacy_WGS84.csv', 10, 11)

In [9]:
df = pd.read_csv('../rawdata/medical_facilities/taiwan_hospital_getcoordinates0812.csv', encoding = 'utf-8')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23302 entries, 0 to 23301
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   欄1      23302 non-null  int64  
 1   機構名稱    23302 non-null  object 
 2   權屬別     23302 non-null  object 
 3   型態別     23302 non-null  object 
 4   地址      23302 non-null  object 
 5   修正地址    23302 non-null  bool   
 6   lon     23193 non-null  object 
 7   lat     23191 non-null  float64
dtypes: bool(1), float64(1), int64(1), object(5)
memory usage: 1.3+ MB


,欄1,機構名稱,權屬別,型態別,地址,修正地址,lon,lat
0,15082,金門縣烈嶼鄉衛生所,衛生所,衛生所,金門縣烈嶼鄉西口村西方6-5號,False,118.2395729,24.435179
1,21791,十方牙醫診所,私立牙醫診所,牙醫診所,金門縣烈嶼鄉西路35-3號,False,118.245518,24.430369
2,19300,安康診所,私立西醫診所,西醫診所,金門縣烈嶼鄉東林街163號,False,118.2484485,24.427585
3,11720,大千牙醫診所,私立牙醫診所,牙醫診所,金門縣金城鎮民權路23號,False,118.3138066,24.431652
4,17554,健康診所,私立西醫診所,西醫診所,金門縣金城鎮民權路27號3F,False,118.313809,24.431726


In [10]:
df = df.dropna(subset=["lon"])
df['lon'] = pd.to_numeric(df['lon'],errors='coerce')
df['lat'] = pd.to_numeric(df['lat'],errors='coerce')
geom = [Point(xy) for xy in zip(df['lon'], df['lat'])]
crs = {'init': 'epsg:4326'}
#crs = {'init': 'epsg:3826'}
gf = gpd.GeoDataFrame(df, crs=crs, geometry=geom)
gf = gf.to_crs(epsg=3826)
gf.info()

C:\Users\tazz4\anaconda3\envs\ox\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 23193 entries, 0 to 23279
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   欄1        23193 non-null  int64   
 1   機構名稱      23193 non-null  object  
 2   權屬別       23193 non-null  object  
 3   型態別       23193 non-null  object  
 4   地址        23193 non-null  object  
 5   修正地址      23193 non-null  bool    
 6   lon       23191 non-null  float64 
 7   lat       23191 non-null  float64 
 8   geometry  23193 non-null  geometry
dtypes: bool(1), float64(2), geometry(1), int64(1), object(4)
memory usage: 1.6+ MB


In [11]:
#gf = gf[['機構名稱', 'geometry']]
#filt = (gf['型態別'].str.contains('中醫', na=False)) | (gf['型態別'].str.contains('西醫', na=False))
filt = (gf['型態別'].str.contains('醫院', na=False))
gf = gf.loc[filt,['機構名稱', '型態別', 'geometry']]
#gf = gf[['中文單位名稱','POINT_X','POINT_Y', 'geometry']]
gf.to_file(f'./medical_facilities/taiwan_hospital_getcoordinates0812' + ".geojson", driver='GeoJSON')